In [1]:
import tensorflow as tf

2025-01-08 22:08:35.963272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
batch_size = 4
q_seq_len = 5
k_seq_len = 6
qk_dim = 8
v_dim = 10
Q = tf.random.normal([batch_size, q_seq_len, qk_dim])
K = tf.random.normal([batch_size, k_seq_len, qk_dim])
V = tf.random.normal([batch_size, k_seq_len, v_dim])

In [5]:
matmul_qk = tf.matmul(Q, K, transpose_b=True)


In [4]:
dk = qk_dim
dk

8

In [8]:
tf.math.sqrt(dk)
# 所以需要把dk转为浮点数

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt] name: 

In [9]:
dk_float = tf.cast(dk, tf.float32)
type(dk_float), type(dk)

(tensorflow.python.framework.ops.EagerTensor, int)

In [12]:
scaled_attention_logits = matmul_qk / tf.math.sqrt(dk_float)
scaled_attention_logits.shape # batch_size, q_seq_len, k_seq_len

TensorShape([4, 5, 6])

In [15]:
attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
attention_weights.shape, V.shape

(TensorShape([4, 5, 6]), TensorShape([4, 6, 10]))

In [16]:
attention_qkv = tf.matmul(attention_weights, V)
attention_qkv.shape

TensorShape([4, 5, 10])

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    # Q: batch_size, q_seq_len, qk_dim
    # K: batch_size, k_seq_len, qk_dim
    # V: batch_size, k_seq_len, v_dim
    matmul_qk = tf.matmul(Q, K, transpose_b=True)
    dk = tf.cast(K.shape[-1], tf.float32)
    scaled_matmul_qk = matmul_qk / tf.math.sqrt(dk)
    attention_weights = tf.nn.softmax(scaled_matmul_qk, axis=-1)
    attention_qkv = tf.matmul(attention_weights, V)
    return attention_qkv